# 1. Import and Install Dependencies

# Run installs

In [4]:
!pip install mediapipe
!pip install tensorflow==2.14.0
!pip install opencv-python
!pip install mediapipe
!pip install matplotlib

# Run preps

In [10]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow import keras
import tensorflow
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.applications.mobilenet_v2 import MobileNetV2
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import multilabel_confusion_matrix, classification_report

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

def rename_dirs(starting_pos):
    all_data_dir_path = os.path.join(os.getcwd(), 'MP_Data')
    
    activities_dirs = [name for name in os.listdir(all_data_dir_path) if os.path.isdir(os.path.join(all_data_dir_path, name))]
    
    if not len(activities_dirs):
        return
    
    # Get the current directory
    for activity_dir_name in activities_dirs:
        banned_names = ["move_left", "move_right", "pc_lock", "pc_on"]
        if activity_dir_name in banned_names:
            continue
        activity_dir_path = os.path.join(all_data_dir_path, activity_dir_name)
    
        # List all directories in the current location
        activity_dir_enumerated_dirs = [num_name for num_name in os.listdir(activity_dir_path) if os.path.isdir(os.path.join(activity_dir_path, num_name))]

    
        if starting_pos > 0:
            activity_dir_enumerated_dirs.sort(reverse=True, key=int)
        else:
            activity_dir_enumerated_dirs.sort(reverse=False, key=int)

        # Rename the directories
        for directory_num_name in activity_dir_enumerated_dirs:
            new_name = int(directory_num_name) + int(starting_pos)
            os.rename(os.path.join(activity_dir_path, str(directory_num_name)), os.path.join(activity_dir_path, str(new_name)))

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

def save_model_as_tflite(base_model):
    #base_model = MobileNetV2(include_top=True, weights='imagenet')
    #base_model = model.load_weights("my_model.keras")
    output_model = "./output.tflite"

    converter = tensorflow.lite.TFLiteConverter.from_keras_model(base_model)
    converter.target_spec.supported_ops = [tensorflow.lite.OpsSet.TFLITE_BUILTINS, tensorflow.lite.OpsSet.SELECT_TF_OPS]
    converter._experimental_lower_tensor_list_ops = False

    tflite_quant_model = converter.convert()
    with open(output_model, 'wb') as o:
        o.write(tflite_quant_model)
        

colors = [(245,117,16), (117,245,16), (16,117,245), (1,100,200), (10,50,70), (1,150,30), (1,150,30), (1,150,30)]

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
# actions = np.array(['lock', 'move_left', 'move_right', 'show_desktop', 'mute', 'unmute'])
actions = np.array(['lock', 'move_left', 'move_right', 'show_desktop', 'mute', 'unmute'])

# Thirty videos worth of data
no_sequences = 120

# Videos are going to be 30 frames in length
sequence_length = 20

# Run test keypoints

In [6]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# Manage folders

In [59]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [63]:
rename_dirs(0)

# Collect Keypoint Data for Training

In [61]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(900)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [8]:
cap.release()
cv2.destroyAllWindows()

# Build and Train LSTM Neural Network

In [7]:
label_map = {label:num for num, label in enumerate(actions)}
sequences, labels = [], []

# Define the number of sequences for each action

for action in actions:
    
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

np.save('sequences.npy', np.array(sequences, dtype=object), allow_pickle=True)
np.save('labels.npy', np.array(labels, dtype=object), allow_pickle=True)

y = to_categorical(labels).astype(int)
X = np.array(sequences)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(20, 258), name='lstm_1'))
model.add(LSTM(128, return_sequences=True, activation='relu', name='lstm_2'))
model.add(LSTM(64, return_sequences=False, activation='relu', name='lstm_3'))
model.add(Dense(64, activation='relu', name='dense_1'))
model.add(Dense(32, activation='relu', name='dense_2'))
model.add(Dense(actions.shape[0], activation='softmax', name='output_layer'))
res = [.7, 0.2, 0.1]
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [66]:
np.array(sequences).shape

(720, 20, 258)

In [67]:
actions.shape[0]

6

In [68]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 20, 64)            82688     
                                                                 
 lstm_2 (LSTM)               (None, 20, 128)           98816     
                                                                 
 lstm_3 (LSTM)               (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 output_layer (Dense)        (None, 6)                 198       
                                                                 
Total params: 237350 (927.15 KB)
Trainable params: 237

In [8]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])


Epoch 1/100
21/21 [==============================] - 6s 52ms/step - loss: 1.3731 - categorical_accuracy: 0.4414
Epoch 2/100
21/21 [==============================] - 1s 33ms/step - loss: 0.5774 - categorical_accuracy: 0.7623
Epoch 3/100
21/21 [==============================] - 1s 46ms/step - loss: 0.4316 - categorical_accuracy: 0.8441
Epoch 4/100
21/21 [==============================] - 1s 50ms/step - loss: 0.2753 - categorical_accuracy: 0.8935
Epoch 5/100
21/21 [==============================] - 1s 34ms/step - loss: 0.1393 - categorical_accuracy: 0.9660
Epoch 6/100
21/21 [==============================] - 1s 37ms/step - loss: 0.1111 - categorical_accuracy: 0.9691
Epoch 7/100
21/21 [==============================] - 1s 33ms/step - loss: 0.5891 - categorical_accuracy: 0.8009
Epoch 8/100
21/21 [==============================] - 1s 35ms/step - loss: 0.2623 - categorical_accuracy: 0.9244
Epoch 9/100
21/21 [==============================] - 1s 35ms/step - loss: 0.8144 - categorical_accuracy:

In [15]:
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
conf_matrix = multilabel_confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)
class_report

3/3 [==============================] - 0s 17ms/step


'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00        15\n           1       1.00      1.00      1.00        13\n           2       1.00      1.00      1.00        14\n           3       0.92      1.00      0.96        11\n           4       1.00      1.00      1.00        10\n           5       1.00      0.89      0.94         9\n\n   micro avg       0.99      0.99      0.99        72\n   macro avg       0.99      0.98      0.98        72\nweighted avg       0.99      0.99      0.99        72\n samples avg       0.99      0.99      0.99        72\n'

In [14]:
import csv

# Replace this with your actual confusion matrices
conf_matrices = [
    [[57, 0], [0, 15]],
    [[59, 0], [0, 13]],
    [[58, 0], [0, 14]],
    [[60, 1], [0, 11]],
    [[62, 0], [0, 10]],
    [[63, 0], [1, 8]]
]

# Specify the file name
csv_file = 'confusion_matrices.csv'

# Write the confusion matrices to the CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Write header
    writer.writerow(['Class', 'TP', 'FN', 'FP', 'TN'])
    
    # Write data
    for i, matrix in enumerate(conf_matrices):
        tp, fn = matrix[0]
        fp, tn = matrix[1]
        writer.writerow([f'Class_{i}', tp, fn, fp, tn])

# Save/Load model 

In [4]:
model.save('my_model.keras')

model.load('my_model.keras')

In [11]:
model = keras.models.load_model('150rc4fun.keras')
save_model_as_tflite(model)

INFO:tensorflow:Assets written to: C:\Users\serwu\AppData\Local\Temp\tmpo9m537m9\assets


INFO:tensorflow:Assets written to: C:\Users\serwu\AppData\Local\Temp\tmpo9m537m9\assets


# Test in Real Time

In [5]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.95

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-20:]
        
        if len(sequence) == 20:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(res)
            #print(actions[np.argmax(res)])
            
            # Calculate recognition percentage
            recognition_percentage = round(100 * res[np.argmax(res)], 2)
            print(f"Gesture recognition percentage: {recognition_percentage}%")
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            #print(res)
            #print(colors)
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


1/1 [==============================] - 1s 532ms/step
[8.4579486e-04 3.0588871e-03 1.6471859e-06 3.6411418e-06 9.9608922e-01
 8.7571442e-07]
Gesture recognition percentage: 99.61%
1/1 [==============================] - 0s 26ms/step
[1.0345832e-03 1.1729789e-02 1.2212246e-05 4.5079625e-05 9.8715973e-01
 1.8568993e-05]
Gesture recognition percentage: 98.72%
1/1 [==============================] - 0s 22ms/step
[9.3188341e-04 4.8138663e-02 9.6713724e-05 1.2534613e-03 9.4909537e-01
 4.8392295e-04]
Gesture recognition percentage: 94.91%
1/1 [==============================] - 0s 23ms/step
[0.00111016 0.13415618 0.00087971 0.04884255 0.8082226  0.00678875]
Gesture recognition percentage: 80.82%
1/1 [==============================] - 0s 23ms/step
[0.00107488 0.13090096 0.00257693 0.4466897  0.39735278 0.02140482]
Gesture recognition percentage: 44.67%
1/1 [==============================] - 0s 24ms/step
[3.1219629e-04 4.1557692e-02 1.5601577e-03 8.8574433e-01 5.6597486e-02
 1.4228224e-02]
Gesture 